In [2]:
# Import libraries
from openmm.app import *
from openmm import *
from openmm.unit import *
from openmmml import MLPotential
import sys

# Setup
pdb = PDBFile("aaa.pdb")
potential = MLPotential('ani2x')
system = potential.createSystem(pdb.topology)

/home/mbowley/miniconda3/envs/ani2/lib/python3.9/site-packages/torchani-2.2.3.dev2+g3dfbaf4-py3.9-linux-x86_64.egg/torchani/resources/


# Works fine on OpenCL

In [2]:
integrator = LangevinIntegrator(
    300 * kelvin, 
    1 / picosecond, 
    1.0 * femtosecond,
)
simulation = Simulation(
    pdb.topology,
    system,
    integrator,
    Platform.getPlatformByName("CPU"),
)
simulation.context.setPositions(pdb.positions)
simulation.minimizeEnergy()
simulation.step(1000)
state_reporter = StateDataReporter(
    sys.stdout,
    100,
    temperature = True,
    potentialEnergy = True,
    speed=True
)
simulation.reporters.append(state_reporter)
print("done")

[W BinaryOps.cpp:601] Warning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (function operator())


KeyboardInterrupt: 

# Breaks with CUDA

In [4]:
integrator = LangevinIntegrator(
    300 * kelvin, 
    1 / picosecond, 
    1.0 * femtosecond,
)
simulation = Simulation(
    pdb.topology,
    system,
    integrator,
    Platform.getPlatformByName("CUDA"),
)
simulation.context.setPositions(pdb.positions)
simulation.minimizeEnergy()
simulation.step(100)
print("done")

OpenMMException: The following operation failed in the TorchScript interpreter.
Traceback of TorchScript, serialized code (most recent call last):
  File "code/__torch__/openmmml/models/anipotential/___torch_mangle_14.py", line 31, in forward
      _5 = torch.mul(boxvectors1, 10.)
      pbc = self.pbc
      _6, energy1, = (model0).forward(_4, _5, pbc, )
                      ~~~~~~~~~~~~~~~ <--- HERE
      energy = energy1
    energyScale = self.energyScale
  File "code/__torch__/torchani/models.py", line 32, in forward
      pass
    aev_computer = self.aev_computer
    species_aevs = (aev_computer).forward(species_coordinates0, cell, pbc, )
                    ~~~~~~~~~~~~~~~~~~~~~ <--- HERE
    neural_networks = self.neural_networks
    species_energies = (neural_networks).forward(species_aevs, None, None, )
  File "code/__torch__/torchani/aev.py", line 68, in forward
        ops.prim.RaiseException("AssertionError: ")
        cell3, pbc0 = _1, _1
      shifts = _0(cell3, pbc0, 5.09

# Troubleshooting

In [2]:
!conda list | grep -E "torch|openmm"

openmm                    7.7.0            py39h792354b_0    conda-forge
openmm-torch              0.5             cuda112py39hb628e3f_0    conda-forge
openmmml                  1.0                      pypi_0    pypi
pytorch                   1.10.0          cuda112py39h3ad47f5_1    conda-forge
pytorch-gpu               1.10.0          cuda112py39h0bbbad9_1    conda-forge
torchani                  2.2.3.dev2+g3dfbaf4          pypi_0    pypi


In [3]:
import torch

print(torch.cuda.is_available())
print(torch.cuda.current_device())
print(torch.cuda.device(0))
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0))

True
0
4
NVIDIA GeForce GTX 1080
